<a href="https://colab.research.google.com/github/saideep3/Test/blob/master/NLP_Assignment_2_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
# Import the pandas library to read our dataset
import pandas as pd

# Get the train/test split package from sklearn for preparing our dataset to
# train and test the model with
from sklearn.model_selection import train_test_split

# Import the numpy library to work with and manipulate the data
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pandas as pd
from keras.utils import to_categorical

import nltk
import random

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [0]:
# import data from google drive
#data = pd.read_csv('train.tsv', sep='\t')
data = pd.read_csv('https://raw.githubusercontent.com/cacoderquan/Sentiment-Analysis-on-the-Rotten-Tomatoes-movie-review-dataset/master/train.tsv', sep='\t')

In [16]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re 

pd.set_option('max_colwidth',400)

wordnet = WordNetLemmatizer()
stopwords_en = stopwords.words("english")
punctuations = "?:!.,;-()"

raw_reviews = data.Phrase.values
cleaned_reviews = []

for i in range(len(raw_reviews)):
  review = str(raw_reviews[i])
  review=re.sub('[^a-zA-Z]',' ',review)
  review=[wordnet.lemmatize(w) for w in word_tokenize(str(review).lower())]
  review=' '.join(review)
  cleaned_reviews.append(review)

data['cleaned_reviews'] = cleaned_reviews

data.head()

,PhraseId,SentenceId,Phrase,Sentiment,cleaned_reviews
0,1,1,"A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .",1,a series of escapade demonstrating the adage that what is good for the goose is also good for the gander some of which occasionally amuses but none of which amount to much of a story
1,2,1,A series of escapades demonstrating the adage that what is good for the goose,2,a series of escapade demonstrating the adage that what is good for the goose
2,3,1,A series,2,a series
3,4,1,A,2,a
4,5,1,series,2,series


In [0]:
X = data.cleaned_reviews.values
#Converting to categorical
Y = to_categorical(data.Sentiment.values)

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

#Intializing Vectorizer
vectorizer = TfidfVectorizer(max_features=4000,stop_words = None, ngram_range=(1,3))
X = vectorizer.fit_transform(X)



In [0]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=2003)

In [0]:
import numpy as np

x_train_np = x_train.toarray()
y_train_np = np.array(y_train)

x_test_np = x_test.toarray()
y_test_np = np.array(y_test)
x_train = np.expand_dims(x_train_np, axis=2)
x_test = np.expand_dims(x_test_np, axis=2)

In [0]:
import torch
from torch.nn import Conv1d
from torch.nn import MaxPool1d
from torch.nn import Flatten
from torch.nn import Linear
from torch.nn.functional import relu
from torch import sigmoid
from torch.utils.data import DataLoader, TensorDataset 

In [0]:
from keras import metrics
from sklearn.metrics import f1_score
#custom metrics
from keras import backend as K
def recall_m(y, y_pred):
    tp = K.sum(K.round(K.clip(y * y_pred, 0, 1)))
    pp = K.sum(K.round(K.clip(y, 0, 1)))
    recall = tp / (pp + K.epsilon())
    return recall

def precision_m(y, y_pred):
    tp = K.sum(K.round(K.clip(y * y_pred, 0, 1)))
    pp = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = tp / (pp + K.epsilon())
    return precision

def f1_m(y, y_pred):
    precision = precision_m(y, y_pred)
    recall = recall_m(y, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, MaxPooling1D, Dropout

#Model Initialization
model = Sequential()

def model_build(data_x):
  #Adding 1st convolution layer
  #with 32 filters and kernel_size = 3
  model.add(Conv1D(filters = 64, kernel_size=3, activation='relu', input_shape=(data_x.shape[1],1)))
  #Adding MaxPool layer with pool_size as 2
  model.add(MaxPooling1D(pool_size =2))
  #Adding 2nd convolution layer
  #with 64 filters and kernel_size = 5
  model.add(Conv1D(filters = 128, kernel_size=5, activation='relu'))
  #Adding MaxPool layer with pool_size as 2
  model.add(MaxPooling1D(pool_size =2))
  #Adding 3rd convolution layer
  #with 128 filters and kernel_size = 7
  model.add(Conv1D(filters = 256, kernel_size=7, activation='relu'))
  #Adding MaxPool layer with pool_size as 2
  model.add(MaxPooling1D(pool_size =2))
  model.add(Flatten())
  #Adding Fully connected layer
  model.add(Dense(128, activation='relu'))
  #Adding Dropout layer
  model.add(Dropout(0.5))
  #Output Layer with 5 outputs
  model.add(Dense(5, activation='softmax')) 






In [23]:
#function call
model_build(x_train_np)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[metrics.categorical_accuracy,f1_m,precision_m, recall_m])
#fit model
history = model.fit(x_train, y_train_np, epochs=10, batch_size=128)


Epoch 1/10
109242/109242 [==============================] - 53s 481us/step - loss: 1.1138 - categorical_accuracy: 0.5556 - f1_m: 0.4659 - precision_m: 0.6616 - recall_m: 0.3638
Epoch 2/10
109242/109242 [==============================] - 52s 474us/step - loss: 0.9731 - categorical_accuracy: 0.6138 - f1_m: 0.5588 - precision_m: 0.6943 - recall_m: 0.4686
Epoch 3/10
109242/109242 [==============================] - 52s 473us/step - loss: 0.9157 - categorical_accuracy: 0.6364 - f1_m: 0.5940 - precision_m: 0.7023 - recall_m: 0.5156
Epoch 4/10
109242/109242 [==============================] - 52s 475us/step - loss: 0.8727 - categorical_accuracy: 0.6557 - f1_m: 0.6211 - precision_m: 0.7128 - recall_m: 0.5511
Epoch 5/10
109242/109242 [==============================] - 52s 476us/step - loss: 0.8407 - categorical_accuracy: 0.6682 - f1_m: 0.6398 - precision_m: 0.7206 - recall_m: 0.5761
Epoch 6/10
109242/109242 [==============================] - 52s 476us/step - loss: 0.8130 - categorical_accuracy: 0

In [24]:
#Running on test set
loss, accuracy, f1_score, precision, recall = model.evaluate(x_test, y_test, verbose=0)
print("Accuracy " + str(accuracy) + ":\n\ff1_score = " + str(f1_score) +
          "\nPrecision = " + str(precision) + "\nRecall = " + str(recall))

Accuracy 0.6462258105856722:
f1_score = 0.6284248004826799
Precision = 0.6761270614979925
Recall = 0.5882780127301466


In [0]:
#Saving model
model.save("1105142_1dconv_reg.h5")


In [26]:

# loading and evaluating  saved model
from numpy import loadtxt
from keras.models import load_model
 
# loading model
model1 = load_model('1105142_1dconv_reg.h5', custom_objects={'f1_m': f1_m, 'precision_m':precision_m, 'recall_m':recall_m})
#Model summary
model1.summary()
model1.evaluate(x_test, y_test_np, batch_size=128)


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_4 (Conv1D)            (None, 3998, 64)          256       
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 1999, 64)          0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 1995, 128)         41088     
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 997, 128)          0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 991, 256)          229632    
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 495, 256)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 126720)           

[0.9278321946195355,
 0.6462258105984033,
 0.6289585777118167,
 0.6760649394742538,
 0.5882780126894068]